In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
import chemprop

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_hi_metrics

def chemprop_prepare_df(original_data):
    result = pd.DataFrame({
        'smiles': original_data['smiles'],
        'targets': original_data['value'].astype(float)
    })
    return result

def chemprop_process_folder(input_path, output_path):
    files = ['train_1.csv', 'train_2.csv', 'train_3.csv', 'test_1.csv', 'test_2.csv', 'test_3.csv']
    for file in files:
        input_data = pd.read_csv(input_path + file)
        output_data = chemprop_prepare_df(input_data)
        output_data.to_csv(output_path + file, index=False)

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/simon/miniconda3/envs/chemprop/lib/python3.8/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/hi/kdr/train_1.csv')
test = pd.read_csv('../../../../data/hi/kdr/test_1.csv')

train

,Unnamed: 0,smiles,value
0,0,Brc1ccc(-c2nc3ccc(Nc4ccnc5ccccc45)cc3[nH]2)cc1,True
1,1064,CCc1ccc(-c2nc3ccc(Nc4ccnc5ccccc45)cc3[nH]2)cc1,False
2,1065,CCc1ccc(-c2nc3ccc(Nc4ncnc5ccccc45)cc3[nH]2)cc1,False
3,3722,COc1ccccc1-c1nc2ccc(Nc3ccnc4ccccc34)cc2[nH]1,False
4,4159,Cc1ccc(-c2nc3ccc(Nc4ccnc5ccccc45)cc3[nH]2)cc1,False
...,...,...,...
495,454,CC(C)S(=O)(=O)c1ccccc1Nc1nc(Nc2cccc(NC(=O)CN)c...,False
496,1850,COC(=O)c1cn2ncnc(Oc3ccc4[nH]c(C)cc4c3F)c2c1C,True
497,4120,Cc1cc2c(F)c(Oc3ncnn4cc(OCCCNS(C)(=O)=O)c(C)c34...,True
498,2979,COc1cc2c(Oc3ccc(N/C=C4\C(=O)NC(=O)N(c5ccc(C)cc...,True


In [4]:
import os
import os
import shutil

temp_datapath = '/tmp/chemprop/'
if os.path.exists(temp_datapath) and os.path.isdir(temp_datapath):
    shutil.rmtree(temp_datapath)
os.mkdir(temp_datapath)

input_paths = [
    '/home/simon/papers/lohi/data/hi/kdr/'
]

output_paths = [
    temp_datapath
]

for i in range(len(input_paths)):
    chemprop_process_folder(input_paths[i], output_paths[i])

In [5]:
base_arguments = [
    '--dataset_type', 'classification', 
    '--data_path', temp_datapath + 'train_1.csv',
    '--separate_val_path', temp_datapath + 'test_1.csv',
    '--separate_test_path', temp_datapath + 'test_1.csv',
    '--save_preds',
    '--metric', 'prc-auc',
    '--epochs', '100',
    '--gpu', '0',
    '--save_dir', temp_datapath + 'checkpoint',
    '--features_generator', 'rdkit_2d_normalized',
    '--no_features_scaling'
]

In [6]:
param_dict = {
    '--depth': ['3', '4', '5', '6'],
    '--dropout': ['0.0', '0.2', '0.3', '0.5', '0.7'],
    '--ffn_hidden_size': ['600', '1200', '2400', '3600'],
    '--ffn_num_layers': ['1', '2', '3'],
    '--hidden_size': ['600', '1200', '2400', '3600']
}

In [7]:
from copy import copy
from tqdm import tqdm
from sklearn.model_selection import ParameterSampler

hyperparam_list = list(ParameterSampler(param_dict, n_iter=20))

best_score = 0.0
best_hyperparams = None

for i, hyperparam in tqdm(enumerate(hyperparam_list)):
    print('HYPERPARAMETER', i)
    print(hyperparam)
    result_arguments = copy(base_arguments)
    for key, value in hyperparam.items():
        result_arguments.append(key)
        result_arguments.append(value)
    # Train
    args = chemprop.args.TrainArgs().parse_args(result_arguments)
    mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

    test_preds = pd.read_csv(temp_datapath + 'checkpoint/test_preds.csv')
    metrics = get_hi_metrics(test, test_preds['targets'].to_list())

    score = metrics['prc_auc']
    if score > best_score:
        print('New best PRC AUC:', score)
        best_score = score
        best_hyperparams = hyperparam

0it [00:00, ?it/s]Command line
python /home/simon/miniconda3/envs/chemprop/lib/python3.8/site-packages/ipykernel_launcher.py --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"d5b0fee2-9f68-4872-9f25-d0fd39dd4883" --shell=9002 --transport="tcp" --iopub=9004 --f=/home/simon/.local/share/jupyter/runtime/kernel-v2-48783Ee3ZnGwS7bEj.json
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache

HYPERPARAMETER 0
{'--hidden_size': '2400', '--ffn_num_layers': '2', '--ffn_hidden_size': '1200', '--dropout': '0.3', '--depth': '5'}


500it [00:00, 417094.67it/s]
100%|██████████| 500/500 [00:00<00:00, 226645.63it/s]
Number of tasks = 1
Fold 0
Splitting data with seed 0
3116it [00:00, 478471.58it/s]
100%|██████████| 3116/3116 [00:00<00:00, 260300.97it/s]
3116it [00:00, 483802.89it/s]
100%|██████████| 3116/3116 [00:00<00:00, 252535.14it/s]
Class sizes
targets 0: 22.20%, 1: 77.80%
Total size = 500 | train size = 500 | val size = 3,116 | test size = 3,116
Building model 0
MoleculeModel(
  (sigmoid): Sigmoid()
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout): Dropout(p=0.3, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=2400, bias=False)
        (W_h): Linear(in_features=2400, out_features=2400, bias=False)
        (W_o): Linear(in_features=2533, out_features=2400, bias=True)
      )
    )
  )
  (readout): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(in_features=2600, out_features=1200, bias=True)
    (2): ReLU()
   

New best PRC AUC: 0.702322029260872
HYPERPARAMETER 1
{'--hidden_size': '2400', '--ffn_num_layers': '2', '--ffn_hidden_size': '1200', '--dropout': '0.7', '--depth': '5'}


Setting molecule featurization parameters to default.
Loading data
500it [00:00, 280367.91it/s]
100%|██████████| 500/500 [00:00<00:00, 146183.74it/s]
Number of tasks = 1
Fold 0
Splitting data with seed 0
3116it [00:00, 253539.44it/s]
100%|██████████| 3116/3116 [00:00<00:00, 266033.98it/s]
3116it [00:00, 471685.12it/s]
100%|██████████| 3116/3116 [00:00<00:00, 268653.41it/s]
Class sizes
targets 0: 22.20%, 1: 77.80%
Total size = 500 | train size = 500 | val size = 3,116 | test size = 3,116
Building model 0
MoleculeModel(
  (sigmoid): Sigmoid()
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout): Dropout(p=0.7, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=2400, bias=False)
        (W_h): Linear(in_features=2400, out_features=2400, bias=False)
        (W_o): Linear(in_features=2533, out_features=2400, bias=True)
      )
    )
  )
  (readout): Sequential(
    (0): Dropout(p=0.7, inplace=False)
    (1): Linear(

...

HYPERPARAMETER 18
{'--hidden_size': '3600', '--ffn_num_layers': '3', '--ffn_hidden_size': '600', '--dropout': '0.3', '--depth': '4'}


500it [00:00, 301965.73it/s]
100%|██████████| 500/500 [00:00<00:00, 245166.24it/s]
Number of tasks = 1
Fold 0
Splitting data with seed 0
3116it [00:00, 499024.49it/s]
100%|██████████| 3116/3116 [00:00<00:00, 257136.00it/s]
3116it [00:00, 501937.60it/s]
100%|██████████| 3116/3116 [00:00<00:00, 269328.84it/s]
Class sizes
targets 0: 22.20%, 1: 77.80%
Total size = 500 | train size = 500 | val size = 3,116 | test size = 3,116
Building model 0
MoleculeModel(
  (sigmoid): Sigmoid()
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout): Dropout(p=0.3, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=3600, bias=False)
        (W_h): Linear(in_features=3600, out_features=3600, bias=False)
        (W_o): Linear(in_features=3733, out_features=3600, bias=True)
      )
    )
  )
  (readout): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(in_features=3800, out_features=600, bias=True)
    (2): ReLU()
    

HYPERPARAMETER 19
{'--hidden_size': '1200', '--ffn_num_layers': '2', '--ffn_hidden_size': '1200', '--dropout': '0.3', '--depth': '5'}


Setting molecule featurization parameters to default.
Loading data
500it [00:00, 143670.07it/s]
100%|██████████| 500/500 [00:00<00:00, 138563.07it/s]
Number of tasks = 1
Fold 0
Splitting data with seed 0
3116it [00:00, 252228.10it/s]
100%|██████████| 3116/3116 [00:00<00:00, 247804.39it/s]
3116it [00:00, 433322.88it/s]
100%|██████████| 3116/3116 [00:00<00:00, 246244.96it/s]
Class sizes
targets 0: 22.20%, 1: 77.80%
Total size = 500 | train size = 500 | val size = 3,116 | test size = 3,116
Building model 0
MoleculeModel(
  (sigmoid): Sigmoid()
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout): Dropout(p=0.3, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=1200, bias=False)
        (W_h): Linear(in_features=1200, out_features=1200, bias=False)
        (W_o): Linear(in_features=1333, out_features=1200, bias=True)
      )
    )
  )
  (readout): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(

New best PRC AUC: 0.7075586126299451


In [9]:
print(best_hyperparams)

{'--hidden_size': '1200', '--ffn_num_layers': '2', '--ffn_hidden_size': '1200', '--dropout': '0.3', '--depth': '5'}


In [10]:
print(best_score)

0.7075586126299451


In [8]:
from copy import copy

result_arguments = copy(base_arguments)
result_arguments.extend([
    '--hidden_size', '1200',
    '--ffn_num_layers', '2',
    '--ffn_hidden_size', '1200',
    '--dropout', '0.3',
    '--depth', '4'
])

args = chemprop.args.TrainArgs().parse_args(result_arguments)
mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)


Command line
python /home/simon/miniconda3/envs/chemprop/lib/python3.8/site-packages/ipykernel_launcher.py --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"5f7ed2af-df14-4584-88a9-9c2d0be5caca" --shell=9002 --transport="tcp" --iopub=9004 --f=/home/simon/.local/share/jupyter/runtime/kernel-v2-28351303lbL6ZQwtmM.json
Args
{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,


 52%|█████▏    | 1632/3116 [00:51<00:50, 29.45it/s]

In [9]:
max_epoch = 1